In [1]:
import pandas as pd
import boto3
import json
import configparser

# Loading the Data Wharehouse Params from Config File

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))


ACCESS_KEY         = config.get('AWS','ACCESS_KEY')
SECRET_KEY         = config.get('AWS','SECRET_KEY')

CLUSTER_TYPE       = config.get("DWH","CLUSTER_TYPE")
NUM_NODES          = config.get("DWH","NUM_NODES")
NODE_TYPE          = config.get("DWH","NODE_TYPE")

CLUSTER_IDENTIFIER = config.get("DWH","CLUSTER_IDENTIFIER")
DB_NAME            = config.get("DWH","DB_NAME")
DB_USER            = config.get("DWH","DB_USER")
DB_PASSWORD        = config.get("DWH","DB_PASSWORD")
DB_PORT            = config.get("DWH","DB_PORT")

IAM_ROLE_NAME      = config.get("DWH", "IAM_ROLE_NAME")

(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAMEDB_NAME", 
                   "DB_USER", "DB_PASSWORD", "DB_PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, 
                   DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, IAM_ROLE_NAME]
             })

FileNotFoundError: [Errno 2] No such file or directory: 'dwh.cfg'

# Creating the Instances fro IAM, EC2,S3, REDSHIFT 

In [3]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY
                   )

iam = boto3.client('iam',aws_access_key_id=ACCESS_KEY,
                     aws_secret_access_key=SECRET_KEY,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY
                       )

NameError: name 'ACCESS_KEY' is not defined

# Check out the sample data sources on S3

In [4]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
    print(obj)
# for obj in sampleDbBucket.objects.all():
#     print(obj)

NameError: name 's3' is not defined


# STEP 1: IAM ROLE

In [5]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
name 'iam' is not defined
1.2 Attaching Policy


NameError: name 'iam' is not defined

# STEP 2: Redshift Cluster

In [6]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

name 'redshift' is not defined


## 2.1 Describe the cluster to see its status

In [7]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'redshift' is not defined

## 2.2 Extracting the Endpoint and ARN role for Redshift

In [8]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

NameError: name 'myClusterProps' is not defined

# STEP 3: Open an incoming TCP port to access the Redshift Cluster Endpoint

In [9]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

name 'ec2' is not defined


# STEP 4:  Verify that we are able to connect to Redshift cluster

In [10]:
%load_ext sql

In [11]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, DWH_ENDPOINT, DB_PORT,DB_NAME)
print(conn_string)
%sql $conn_string

NameError: name 'DB_USER' is not defined

# Delete the Redshift cluster 

In [12]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

NameError: name 'redshift' is not defined

In [13]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'redshift' is not defined

In [14]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)
#### CAREFUL!!

NameError: name 'iam' is not defined